In [3]:
%load_ext autoreload
%autoreload 2
import sys, os, time, json, re
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

from func_tools import import_px_data, standardize, fetch_s3_trade_files, cnn_data_reshaping, reshape_lob_levels, back_to_labels, intraday_vol_ret 
import visualization_tools as viz_t
from labelling_class import Labels_Generator, cleaned_labels, label_insights, get_strategy_pnl

import inspect

import plotly_express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import tensorflow as tf
from keras.utils import np_utils
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Reshape, Conv2D, LSTM, Dense, MaxPooling2D, BatchNormalization, LeakyReLU, concatenate, add, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using TensorFlow backend.


In [6]:
import LOBData

In [7]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict tf to only allocate xGB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=6024)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)
tf.test.gpu_device_name()

''

## Data

In [8]:
# other inputs
length = 100
batch_size = 64

start_plot = 0
end_plot = 30000

In [9]:
%%time
experiments_folder = 'Experiments'
frequency = timedelta(seconds=10)
pair = 'BTC_AAVE'
date_start = '2020_11_01'
date_end = '2021_04_07'
lob_depth = 10
norm_type = 'dyn_z_score'
roll = 7200 * 6
label_technique = 'three_steps'



CPU times: user 5 µs, sys: 4 µs, total: 9 µs
Wall time: 12.9 µs


In [11]:
%%time
raw_data_path = f'./S3_data'
caching_folder = f'{experiments_folder}/cache' 
cache_data = LOBData.LOBData(raw_data_path, pair, caching_folder, frequency = timedelta(seconds=10), levels=10)

cache_data.get_data()

onal data points in Experiments/cache/BTC_AAVE/10_levels/original_frequency/2020-12-16.csv.gz
Generating Experiments/cache/BTC_AAVE/10_levels/10s/2020-12-17.csv.gz
75198 additional data points in Experiments/cache/BTC_AAVE/10_levels/original_frequency/2020-12-17.csv.gz
Generating Experiments/cache/BTC_AAVE/10_levels/10s/2020-12-18.csv.gz
75107 additional data points in Experiments/cache/BTC_AAVE/10_levels/original_frequency/2020-12-18.csv.gz
Generating Experiments/cache/BTC_AAVE/10_levels/10s/2020-12-19.csv.gz
74242 additional data points in Experiments/cache/BTC_AAVE/10_levels/original_frequency/2020-12-19.csv.gz
Generating Experiments/cache/BTC_AAVE/10_levels/10s/2020-12-20.csv.gz
73537 additional data points in Experiments/cache/BTC_AAVE/10_levels/original_frequency/2020-12-20.csv.gz
Generating Experiments/cache/BTC_AAVE/10_levels/10s/2020-12-21.csv.gz
74828 additional data points in Experiments/cache/BTC_AAVE/10_levels/original_frequency/2020-12-21.csv.gz
Generating Experiments/cac

,Unnamed: 0,Datetime,Level,Ask_Price,Ask_Size,Bid_Price,Bid_Size,Sequence
npartitions=148,,,,,,,,
,int64,object,int64,float64,float64,float64,float64,int64
,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...


In [ ]:
# import data
train_dyn_df, test_dyn_df, top_ob_train, top_ob_test = import_px_data(experiments_folder, frequency, pair, date_start, date_end, lob_depth, norm_type, roll)

In [ ]:
# reshape data
# train
train_depth_dyn, train_dt_index_dyn = reshape_lob_levels(train_dyn_df.reset_index(), output_type='array') # 1 train dataset
mid_px_train_dyn = pd.Series((train_depth_dyn[:,2] + train_depth_dyn[:,0]) / 2, index=train_dt_index_dyn) # 2
px_ts_train = top_ob_train.reset_index()[['Mid_Price']]

# test
test_depth_dyn, test_dt_index_dyn = reshape_lob_levels(test_dyn_df.reset_index(), output_type='array') # 1 test dataset
mid_px_test_dyn = pd.Series((test_depth_dyn[:,2] + test_depth_dyn[:,0]) / 2, index=test_dt_index_dyn) # 2
px_ts_test = top_ob_test.reset_index()[['Mid_Price']]

In [12]:
# get trades

### Trades

In [229]:
fetch_s3_trade_files('S3_data', 'Experiments/input', 'USDT_BTC', timedelta(seconds=10))

reading trade files
saving file at Experiments/input/USDT_BTC-trades-10s-2020_04_01-2020_12_31.csv.gz


,globalTradeID,tradeID,date,type,rate,amount,total,orderNumber
0,456641156,33111689,2020-04-01 00:00:59,buy,6420.617001,0.000543,3.486202,519637736227
1,456641153,33111688,2020-04-01 00:00:58,sell,6420.505196,0.012052,77.377810,519637716247
2,456641152,33111687,2020-04-01 00:00:56,sell,6419.109892,0.100000,641.910989,519637697266
3,456641149,33111686,2020-04-01 00:00:56,buy,6420.938504,0.000543,3.486634,519637688275
4,456641144,33111685,2020-04-01 00:00:54,buy,6420.505169,0.022823,146.534676,519637663300
...,...,...,...,...,...,...,...,...
24302,505065984,39512256,2020-12-31 23:59:14,buy,28940.838661,0.000090,2.604675,827670743878
24303,505065981,39512255,2020-12-31 23:59:12,buy,28927.499553,0.247920,7171.705689,827670700921
24304,505065968,39512254,2020-12-31 23:59:02,buy,28917.664350,0.155991,4510.895669,827670615007
24305,505065967,39512253,2020-12-31 23:59:00,sell,28917.145955,0.000115,3.332990,827670595027


In [30]:
# df_quotes = pd.read_csv(f'Experiments/input/USDT_BTC--10lev--10sec--2020_04_04--2021_01_03.csv.gz', index_col=0)
# df_quotes['Datetime'] = pd.to_datetime(df_quotes['Datetime'])

In [213]:
s3_trades_folder = 'S3_data/trades'
pair = 'USDT_BTC'



In [231]:
trades = pd.read_csv('Experiments/input/USDT_BTC-trades-10s-2020_04_01-2020_12_31.csv.gz')

In [ ]:
df_trades.groupby(pd.Grouper(key='date', freq='D')).agg({'total':'sum'})

In [181]:
%%time
df_trades['date'] = pd.to_datetime(df_trades['date'])
df_trades_grp = df_trades.groupby([pd.Grouper(key='date', freq='10s', dropna=False), 'type']).agg({'amount':'sum', 'rate':'mean'}).reset_index()
df_trades_piv = df_trades_grp.pivot(values=['amount', 'rate'], columns='type',index='date').reset_index()

df_trades_piv.columns = list(map("_".join, df_trades_piv.columns)) # "flatten" column names
df_trades_piv.rename(columns={'date_':'Datetime', 'amount_buy':'Ask_Size', 'amount_sell':'Bid_Size', 'rate_buy':'Ask_Price', 'rate_sell':'Bid_Price'}, inplace=True)

# fill gaps with no trades
date_range_reindex = pd.DataFrame(pd.date_range(df_trades_piv['Datetime'].min(), df_trades_piv['Datetime'].max(), freq="10s"), columns=['Datetime'])
df_trades_piv = pd.merge(df_trades_piv, date_range_reindex, right_on='Datetime', left_on='Datetime', how='right')

# impute NAs - zero for size and last px for price
df_trades_piv.loc[:,['Ask_Size', 'Bid_Size']] = df_trades_piv.loc[:,['Ask_Size', 'Bid_Size']].fillna(0)
df_trades_piv.loc[:,['Ask_Price', 'Bid_Price']] = df_trades_piv.loc[:,['Ask_Price', 'Bid_Price']].fillna(method='ffill')


df_trades_piv['Level'] = -1

CPU times: user 3.82 s, sys: 488 ms, total: 4.31 s
Wall time: 4.31 s


In [182]:
%%time
df_all = pd.concat([df_trades_piv, df_quotes]).sort_values(by=['Datetime', 'Level'])

CPU times: user 4.84 s, sys: 1.19 s, total: 6.04 s
Wall time: 6.05 s


In [183]:
df_stdz = df_all[(df_all['Datetime']>='2020-01-01 00:00:00')&(df_all['Datetime']<='2020-12-31 23:59:50')]
df_stdz.set_index(['Datetime','Level'], inplace=True)

In [184]:
%%time
norm_df_size = standardize(df_stdz[['Ask_Size', 'Bid_Size']], 10+1, 'dyn_z_score', roll)
norm_df_price = standardize(df_stdz[['Ask_Price', 'Bid_Price']], 10+1, 'dyn_z_score', roll)

rolling window = 950400, calculate as roll: 43200 * levels: 11 * shape[1]: 2
done
rolling window = 950400, calculate as roll: 43200 * levels: 11 * shape[1]: 2
done
CPU times: user 46.3 s, sys: 10.6 s, total: 56.9 s
Wall time: 56.9 s


In [190]:
df_stdz.describe()

,Ask_Size,Bid_Size,Ask_Price,Bid_Price,Unnamed: 0.1,Sequence
count,2.596320e+07,2.596320e+07,2.596320e+07,2.596320e+07,2.358720e+07,2.358720e+07
mean,7.529761e-01,6.687193e-01,1.203929e+04,1.202854e+04,4.319950e+04,9.505883e+08
std,1.284103e+00,1.254996e+00,4.452705e+03,4.444519e+03,2.494153e+04,1.180569e+08
min,0.000000e+00,0.000000e+00,6.153252e+03,6.152474e+03,0.000000e+00,7.138388e+08
25%,4.268000e-02,3.382259e-02,9.275622e+03,9.270314e+03,2.159975e+04,8.634625e+08
50%,2.000000e-01,1.659653e-01,1.067745e+04,1.067097e+04,4.319950e+04,9.581165e+08
75%,9.782285e-01,7.265500e-01,1.299600e+04,1.298473e+04,6.479925e+04,1.049790e+09
max,1.125158e+02,1.347703e+02,2.944634e+04,2.930673e+04,8.639900e+04,1.143096e+09


## Labels

In [ ]:
# train labels
mid_px_train = px_ts_train['Mid_Price']
labels_train, smoothed_px_train, df_trades_train = cleaned_labels(mid_px_train, method=label_technique, print_details=False)

# encode
encoded_train_labels = np_utils.to_categorical(labels_train.values,3) 

# # get transaction df
# strategy_df_train = get_strategy_pnl(mid_px_train, labels_train)

viz_t.plot_labels_line(mid_px_train[start_plot:end_plot], 
    labels_train[start_plot:end_plot], 
    title='Train Labels', 
    smoothed_signal=smoothed_px_train[start_plot:end_plot])

In [ ]:
# test labels
mid_px_test = px_ts_test['Mid_Price']
labels_test, smoothed_px_test, df_trades_test = cleaned_labels(mid_px_test, method=label_technique, print_details=False)

# encode
encoded_test_labels = np_utils.to_categorical(labels_test.values,3) 

# # get transaction df
# strategy_df_test = get_strategy_pnl(mid_px_test, labels_test)

viz_t.plot_labels_line(mid_px_test[start_plot:end_plot], 
    labels_test[start_plot:end_plot], 
    title='Test Labels', 
    smoothed_signal=smoothed_px_test[start_plot:end_plot])

## Visual checks

In [ ]:
# viz_t.plot_trades_distribution(df_trades_train[df_trades_train['cleaned_labels']!=0], bin_size=0.0001, metric='gross_returns', fig_width=900, fig_height=550)

In [ ]:
#viz_t.plot_trades_length_overview(df_trades_train[df_trades_train['cleaned_labels']!=0], x='trade_len',  y='gross_returns')

In [ ]:
# fig = px.histogram()
# fig.add_trace(go.Histogram(x=df_trades_train['trade_len'].values, name='train', autobinx = False, xbins={'size':5}))
# fig.add_trace(go.Histogram(x=df_trades_test['trade_len'].values, name='test', autobinx = False, xbins={'size':5}))

# # The two histograms are drawn on top of another
# fig.update_layout(barmode='overlay')
# fig.update_traces(opacity=0.75)
# fig.show()

In [ ]:
viz_t.plot_timeseries(ts_list=[top_ob_train.set_index('Datetime')['Mid_Price'], top_ob_test.set_index('Datetime')['Mid_Price'], mid_px_train_dyn, mid_px_test_dyn], primary_axis=[True, True, False, False], legend=['train-px', 'test-px', 'train-dyn', 'test-dyn'], sample_size=180)

## Model Setup

In [ ]:
def create_light_deeplob(T, lob_depth):
    ## just a test

    input_lmd = Input(shape=(T, lob_depth * 4, 1))
    conv_first1 = Conv2D(16, (1, 2), strides=(1, 2))(input_lmd)
    conv_first1 = LeakyReLU(alpha=0.01)(conv_first1)    
    conv_first1 = BatchNormalization()(conv_first1)
    # conv_first1 = Dropout(.5)(conv_first1)
    
    # note on learnable parameters: CONV2(filter shape =1*2, stride=1) layer is: ((shape of width of filter * shape of height filter * number of filters in the previous layer+1) * number of filters) = 2080 or ((2*1*32)+1)*32
    conv_first1 = Conv2D(16, (1, 2), strides=(1, 2))(conv_first1)
    conv_first1 = LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = BatchNormalization()(conv_first1)

    conv_first1 = Conv2D(16, (1, lob_depth))(conv_first1)
    conv_first1 = LeakyReLU(alpha=0.01)(conv_first1)
    conv_first1 = BatchNormalization()(conv_first1)
    print(conv_first1.shape)

    convfirst_output = Reshape((int(conv_first1.shape[1])* int(conv_first1.shape[3]),))(conv_first1)
    print(convfirst_output.shape)
    # note on learnable parameters:FC3 layer is((current layer c*previous layer p)+1*c) with c being number of neurons
    out = Dense(3, activation='softmax')(convfirst_output)
    print(out.shape)
    model = Model(inputs=input_lmd, outputs=out)
    adam = Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-07)
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

model_code = inspect.getsource(create_light_deeplob)
lines_with_short_desription = [line for line in model_code.split('\n') if "##" in line]
short_description = re.sub(r'\W+', '_', lines_with_short_desription[0])

create_light_deeplob(length, lob_depth).summary()

In [ ]:
date_time_now = datetime.now().strftime("%y%m%d-%H%M%S")
experiment_id = f'{date_time_now}-{pair}-{frequency.seconds}s-{lob_depth}l-{length}-{date_start}-{date_end}{short_description}'
results_folder = f'{experiments_folder}/{pair}/{experiment_id}'
os.makedirs(f'{results_folder}', exist_ok=True)
batch_size=256

config = {
  'pair': pair,
  'frequency': frequency.seconds,
  'lob_depth': lob_depth,
  'length': length,
  'date_start': date_start,
  'date_end': date_end,
  'norm_type': norm_type,
  'roll': roll,
  'batch_size': batch_size,
  'label_technique': label_technique
#   'min_profit': min_profit,
#   'k_plus': k_plus,
#   'k_minus': k_minus,
#   'alpha': alpha,
#   'trading_fee': trading_fee,

#   'input': input_file_name,
#   'normalized_train_file': normalized_train_file,
#   'normalized_test_file':   normalized_test_file,
#   'top_ob_train_file': top_ob_train_file,
#   'top_ob_test_file': top_ob_test_file
}

with open(f'{results_folder}/config.json', 'w') as fp:
    json.dump(config, fp, default=str)

with open(f'{results_folder}/model_code.py', 'w') as fp:
    fp.write(model_code)

light_deeplob = create_light_deeplob(length, lob_depth)
with open(f'{results_folder}/model_summary.txt', 'w') as fp:
    light_deeplob.summary(print_fn=lambda x: fp.write(x + '\n'))


In [ ]:
# try to train the model on smoother version of the data

## Training

In [ ]:
light_deeplob = create_light_deeplob(length, lob_depth)

model_checkpoint_path = f'{results_folder}/{experiment_id}.h5'

# Learning rate callback. Reduce on Plateau multiply the lr by the factor if val loss does not improve for n epochs (patience)
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                   factor=0.2, 
                                                   patience=20)

# Checkpoint callback. Saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(model_checkpoint_path,
                                                 save_best_only=True,
                                                 save_weights_only=False,
                                                 verbose=2,
                                                 save_freq='epoch') # every epoch

# Early stopping callback. When sees no progress on the validation set
es_callback = tf.keras.callbacks.EarlyStopping(patience=50,
                                               restore_best_weights=True)

# Tensorboard callback
tb_callback = tf.keras.callbacks.TensorBoard(results_folder)

# Train and Test time series generators
generator_train = TimeseriesGenerator(
    train_depth_dyn,
    encoded_train_labels,
    length,
    batch_size=batch_size,
    shuffle=True
)

# to be replaced with validation?
generator_test = TimeseriesGenerator(
    test_depth_dyn,
    encoded_test_labels,
    length,
    batch_size=batch_size,
    shuffle=True
)


# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

In [ ]:
# model_name = '/home/federico/Python_vsc_dir/RL_Trader/Experiments/USDT_BTC/210119-184504-USDT_BTC-10s-10l-300-2020_04_04-2021_01_03_binary_classification_full_inception_lighter_deep_lob_model_with_longer_timesteps_300_/210119-184504-USDT_BTC-10s-10l-300-2020_04_04-2021_01_03_binary_classification_full_inception_lighter_deep_lob_model_with_longer_timesteps_300_.h5'
# loaded_light_deep_lob = tf.keras.models.load_model(model_name)


In [ ]:
# Train the model
light_deeplob.fit(generator_train, 
            epochs=200, 
            verbose=0,
            validation_data=generator_test,
            callbacks=[lr_callback, cp_callback, es_callback, tb_callback])

## Evaluating

In [ ]:
model_name = '/home/federico/Python_vsc_dir/RL_Trader/Experiments/USDT_BTC/210221-200759-USDT_BTC-10s-10l-100-2020_04_04-2021_01_03_big_lr_big_batch_size_16_filter_size_shuffle/210221-200759-USDT_BTC-10s-10l-100-2020_04_04-2021_01_03_big_lr_big_batch_size_16_filter_size_shuffle.h5'

In [ ]:
# Load the previously saved weights and evaluate model performance
deep_lob_loaded = tf.keras.models.load_model(model_name)
generator_test = TimeseriesGenerator(
    test_depth_dyn,
    encoded_test_labels,
    length,
    batch_size=batch_size,
    shuffle=False
)

def evaluate_model(model):
    # Re-evaluate the model
    loss, acc = model.evaluate(generator_test, verbose=2)
    print("Restored model, accuracy: {:5.2f}%".format(100*acc))

#evaluate_model(deep_lob_loaded)

In [ ]:
# Get predicted labels
predictions_prob = deep_lob_loaded.predict(generator_test, verbose=1)
map_labels = np.vectorize(back_to_labels) # vectorize back to labels from func_tools
predicted_labels = pd.Series(map_labels(np.argmax(predictions_prob,axis=1)), name='predicted_labels')

In [ ]:
# Experimental: predicted labels on rolling avg
predictions_prob_wa = pd.DataFrame(predictions_prob).rolling(window=10).mean().values
map_labels = np.vectorize(back_to_labels) # vectorize back to labels from func_tools
predicted_labels_wa = pd.Series(map_labels(np.argmax(predictions_prob_wa,axis=1)), name='predicted_labels_wa') # back to original 1,0,-1

In [ ]:
print('##### Predicted labels #####')
label_insights(predicted_labels)


In [ ]:
print('##### Weighted average predicted labels #####')
label_insights(predicted_labels_wa)

In [ ]:
predicted_labels.shape, test_depth_dyn.shape

In [ ]:
# dangerous assigning offset here, wrap it into a function
# align prediction with "length" offset
index_range = np.arange(length, predicted_labels.shape[0] + length) # offset ts length fed to ts generator
predicted_labels.index = index_range

# generate timeseries with buy, sell, zero prob
buy_prob = pd.Series(predictions_prob[:,1], index=index_range)
sell_prob = pd.Series(predictions_prob[:,2], index=index_range)
zero_prob = pd.Series(predictions_prob[:,0], index=index_range)

buy_prob_wa = pd.Series(predictions_prob_wa[:,1], index=index_range)

viz_t.plot_labels_line(top_ob_test['Mid_Price'][start_plot:end_plot], 
    labels_test[start_plot:end_plot], # original labels
    title='Train Set Labels', 
    #smoothed_signal=smoothed_px_test[start_plot:end_plot],
    predicted_labels=predicted_labels[start_plot:end_plot],
    buy_prob_labels=buy_prob[start_plot:end_plot],
    #sell_prob_labels=sell_prob[start:end],
    predictions_prob_wa=buy_prob_wa[start_plot:end_plot],
    width=1100, height=600
    )

In [ ]:
### DOUBLE CHECK that labels and px_ts are correctly aligned
px_ts = top_ob_test['Mid_Price']# adjust prediction offsset
datetime_ts = top_ob_test['Datetime']
trades_timeseries = get_strategy_pnl(px_ts, predicted_labels)
df_trades = trades_timeseries.dropna(subset=['gross_returns'])

In [ ]:
df_trades

In [ ]:
# px_ts = top_ob_test['Mid_Price'][length:].reset_index()['Mid_Price']# adjust prediction offsset
# datetime_ts = top_ob_test['Datetime'][length:].reset_index()['Datetime']
# trades_timeseries = get_strategy_pnl(px_ts, predicted_labels)
# df_trades = trades_timeseries.dropna(subset=['gross_returns'])

In [ ]:
### to do:
# seek for patterns in prediction probability
# day vs night - weekday vs weekend - model certainty before long trades vs short trades
# determine if predictions are naive